# Lab 1 - Creating the SQL Tables

In this lab, use `sqlalchemy` to create, populate, and query a table from the baseball database, as well as for the `super_hero_powers.csv` table.  

In [1]:
import pandas as pd
artwork = pd.read_csv("./data/Artworks.csv")

## Part 1 - Baseball Managers

In this part of the lab, you will walk through the process of creating a manager table from [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/)

## Task 1 - Download, unzip, rename 

1. Download the baseball database linked above (save to desktop)
2. Unzip the file and rename to `baseball`
3. Load the `core/Managers.csv` file into a pandas `DataFrame` using `read_csv`
4. Inspect the `column` names and `dtypes`

In [2]:
baseball = pd.read_csv("./data/baseballdatabank/core/Managers.csv")

In [5]:
baseball.columns

Index(['playerID', 'yearID', 'teamID', 'lgID', 'inseason', 'G', 'W', 'L',
       'rank', 'plyrMgr'],
      dtype='object')

In [7]:
baseball.dtypes

playerID     object
yearID        int64
teamID       object
lgID         object
inseason      int64
G             int64
W             int64
L             int64
rank        float64
plyrMgr      object
dtype: object

In [4]:
baseball.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y


#### Task 2 - Create a `sqlalchemy` types `dict`

In [10]:
from sqlalchemy import Integer, Float, String

sql_types = {'playerID': String,
            'yearID': Integer,
            'teamID': String,
            'lgID': String,
            'inseason': Integer,
            'G': Integer,
            'W': Integer,
            'L':Integer,
            'rank': Float,
            'plyrMgr': String}

#### Task 4 - Create an `engine` and `schema`

In [11]:
!rm databases/baseball.db

rm: databases/baseball.db: No such file or directory


In [15]:
from sqlalchemy import create_engine

In [18]:
engine = create_engine("sqlite:///base.db", echo = False)

In [26]:
schema = pd.io.sql.get_schema(baseball, 'base', con = engine, dtype = sql_types)
schema

'\nCREATE TABLE base (\n\t"playerID" VARCHAR, \n\t"yearID" INTEGER, \n\t"teamID" VARCHAR, \n\t"lgID" VARCHAR, \n\tinseason INTEGER, \n\t"G" INTEGER, \n\t"W" INTEGER, \n\t"L" INTEGER, \n\trank FLOAT, \n\t"plyrMgr" VARCHAR\n)\n\n'

#### Task 5 - Use `to_sql` with `if_exists='append'` to insert the data

In [32]:
baseball.to_sql('base',
                con=engine,
                dtype = sql_types,
                index = False,
                if_exists = 'append')

#### Task 6 - Query the table to make sure it all worked

In [66]:
from sqlalchemy.ext.automap import automap_base
Base = automap_base()

In [67]:
Base.prepare(engine2, reflect=True)

In [68]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

Base= Base.metadata.tables['base']
Base


Table('base', MetaData(bind=None), Column('playerID', VARCHAR(), table=<base>), Column('yearID', INTEGER(), table=<base>), Column('teamID', VARCHAR(), table=<base>), Column('lgID', VARCHAR(), table=<base>), Column('inseason', INTEGER(), table=<base>), Column('G', INTEGER(), table=<base>), Column('W', INTEGER(), table=<base>), Column('L', INTEGER(), table=<base>), Column('rank', FLOAT(), table=<base>), Column('plyrMgr', VARCHAR(), table=<base>), schema=None)

In [62]:
session = Session()

In [63]:
from sqlalchemy import select
stmt = select('*').select_from(Base)


In [64]:
session.execute(stmt).fetchmany(5)

[('wrighha01', 1871, 'BS1', None, 1, 31, 20, 10, 3.0, 'Y'),
 ('woodji01', 1871, 'CH1', None, 1, 28, 19, 9, 2.0, 'Y'),
 ('paborch01', 1871, 'CL1', None, 1, 29, 10, 19, 8.0, 'Y'),
 ('lennobi01', 1871, 'FW1', None, 1, 14, 5, 9, 8.0, 'Y'),
 ('deaneha01', 1871, 'FW1', None, 2, 5, 2, 3, 8.0, 'Y')]

## Part 2 - Super Hero Powers

Now make a database and table for the super hero powers.

## Problem 1
    
**Task:** One the `super_hero_powers.csv` and verify that the contents of the columns are all Boolean.  In this problem, you need to

1. Create a `dict` that defines the `pandas` column type
2. Read the file in using a `pd.read_csv`.
3. Clean up all the column labels.
    
**Be sure to write clean code!**


In [56]:
hero = pd.read_csv('./data/super_hero_powers.csv')
hero.head()

,hero_names,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Problem 2
    
Now define an `sqlalchemy` table for these data using `pandas` `to_sql` dataframe method.  You can use the `sqlalchemy.String` and `sqlalchemy.Boolean` columns type, which are [documented here](https://docs.sqlalchemy.org/en/latest/core/type_basics.html)

## Problem 3
    
Now you need to make a new `engine`, `inspect` your database, and make a `session` to query the database.

## Problem 4
    
Perform `sqlalchemy` queries to answer each of the following questions.

1. How many heroes have both Super Strength and Super Speed?
2. How many heroes have names that start with the word *Black*
3. Are heroes with Agility more likely to have Stealth?
4. What fraction of all heroes that can fly also have Super Strength?
5. Consider heroes that have names that contain `"girl"`, `"boy"`, `"woman"`, or `"man"`.  Compute the following ratio

$$\frac{N(\text{boy or man})}{N(\text{girl or woman}}$$

**Hint:** You will need to use some combination of `where`, `group_by`, and `count` for each part.

## Problem 5

Tell me another cool fact about the super powers.